# Schnittstellen von Twitter

Semesterarbeit 1

## EDS-Einführung in Data Science

Klasse: BSc INF-P-IN010, BE1, HS20/21<br>
Dozent: Dr. Tim vor der Brück<br>
Autor:  Sandro Bürki, Michael Friderich<br>
Datum:  05.09.2020<br>

## Abstract / Executive Summary

In diesem ersten Teil der Semesterarbeit für das Modul EDS - Einführung in Data Science werden vier Fragen zum
Nachrichtendienst Twitter beantwortet. Dabei stehen die Kommunikationsschnittstelle API, die Systemarchitektur und die
Datenmengen im Vordergrund. Zuerst wird der Zugriff auf die API-Schnittstelle mittels Python Code vorgeführt.
Dabei werden auch die Einschränkungen des Zugriffs auf die API-Schnittstelle aufgezeigt. Die Benutzeranfragen werden
durch die Systemarchitektur abgearbeitet, wobei mehrere Load Balancer die Anfragen verteilen. Dabei geschieht
entweder ein Fan-Out bei Benutzern mit wenigen Followern oder ein Speichern des Tweets bei Benutzern mit sehr vielen
Followern. Um diese Datenmengen bewältigen zu können werden für die spezifischen Use-Cases relationale sowie NoSQL
Datenbanken eingesetzt.

## Einleitung

In diesem ersten Teil der Semesterarbeit im Modul EDS - Einführung in Data Science, werden die folgenden Fragen zur
Kommunikationsschnittstelle API, der Systemarchitektur und der Datenmenge beantwortet.
1. Wie kann man auf einen Twitter Stream zugreifen - zeigen Sie, wie Sie Ihre eigenen Topics abrufen.
2. Welche Einschränkungen hat das Twitter API?
3. Wie werden die Anfragen in der Systemlandschaft von Twitter abgearbeiteten (nur auf Ebene Systemarchitektur)
4. Welchen Stellenwert haben NoSQL Datenbanken bei der Durchführung dieser Aufgabe?

## Materialien und Methoden

Einige der Fragestellungen dieser Arbeit sind nicht direkt wissenschaftlicher Natur, weshalb sich die Methodik dieser
Arbeit von einem wissenschaftlichen Artikel unterscheiden wird. Generell dienen das Lehrbuch Mining the Social Web
(Russell/Klassen 2019), sowie die offizielle Twitter-Dokumentation (Twitter (o. J.): Twitter API v1.1) als primäre
Informationsquellen.

EIn Twitter Stream ermöglicht den Zugriff auf Tweets in Echtzeit. Diese Streams können z. B. nach Stichwörtern, Sprache,
und Ort gefiltert werden. Mithilfe des Codes in Listing 1 wird eine Suchabfrage abgesetzt, die uns aktuelle Tweets für
das Stichwort “Open Source” anzeigt.

Um Zugriff auf die Twitter API zu erhalten, wird ein Developer Account benötigt. Mithilfe dessen lassen sich die
Zugangsdaten für eine automatisierte, programmierbare Datenabfrage generieren. Diese Schlüssel können im oben
aufgeführten Python Code an den angegebenen Platzhaltestellen ​ <key> eingesetzt werden. Nach dem erfolgreichen
Login wird der/die gewünschte/n Suchbegriffe abgefragt, die aktuellen Tweets abgerufen und ausgegeben. Die Abfrage
wird weitergeführt bis Sie manuell abgebrochen oder durch andere Verbindungsunterbrüche beendet wird. Hierbei müsste
das Programm und somit die Connection erneut gestartet werden.

Um die Fragestellung bezüglich dem Stellenwert von NoSQL Datenbanken beantworten zu können, müssen die Anforderungen
von Twitter mit den Unterschieden sowie den Vor- und Nachteilen von NoSQL gegenüber klassischen, relationalen Datenbanken
verglichen werden.

In [12]:
import twitter
import Twitter_credentials

def oauth_login():
    auth = twitter.oauth.OAuth(Twitter_credentials.OAUTH_TOKEN,
                               Twitter_credentials.OAUTH_TOKEN_SECRET,
                               Twitter_credentials.CONSUMER_KEY,
                               Twitter_credentials.CONSUMER_SECRET)

    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()
print(twitter_api)

In [13]:
# Finding topics of interest by using the filtering capabilities it offers.
# Query terms

import sys

q = 'Open Source' # Comma-separated list of terms

print('Filtering the public timeline for track={0}'.format(q), file=sys.stderr)
sys.stderr.flush()

Filtering the public timeline for track=Open Source


In [14]:
# Returns an instance of twitter.Twitter
twitter_api = oauth_login();

In [15]:
# Reference the self.auth parameter
twitter_stream = twitter.TwitterStream(auth=twitter_api.auth);

In [16]:
# See https://developer.twitter.com/en/docs/tutorials/consuming-streaming-data
streamm = twitter_stream.statuses.filter(track=q)

TwitterHTTPError: Twitter sent status 420 for URL: 1.1/statuses/filter.json using parameters: (oauth_consumer_key=hhff9myUkucabaPajxuveAGHL&oauth_nonce=13288381118041823390&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1610136281&oauth_token=1294354836662693888-atG4VxUdebbTSfCOQ8vcbB3f6owo5Y&oauth_version=1.0&track=Open%20Source&oauth_signature=PDklvEs61o1pQqs9Y39ro%2BN6Dko%3D)
details: Exceeded connection limit for user


In [17]:
for tweet in twitter.stream:
    print(tweet['text'])
    sys.stdout.flush()

# Save to a database in a particular collection

TypeError: 'module' object is not iterable

## Ergebnisse / Findings / Resultate

Mittels dem oben vorgestellten Python Code können Tweets anhand von Stichwörter abgefragt werden. Dadurch erhalten
wir live dazu passende Tweets. Wir können nun den Suchbegriff verändern und sehen wie häufig neue Tweets erstellt
werden. Bei prominenten Stichwörtern wie z. B. Justin Bieber und Donald Trump erhalten wir sehr viel neue Tweets bei
weniger häufig genutzten Stichwörtern wie z. B. Open Source ist die Zeitspanne zwischen zwei Tweets deutlich grösser.

Bei jeglichen Abfragen gelten die Twitter Nutzungsbedingungen für Entwickler (Twitter (o. J.): Developer Agreement and
Policy). Unter anderem umfassen diese Off-Twitter Matching, Spamming und legale Vorgaben. Für eine optimale Nutzung des
Dienstes in einer kommerziell tauglichen Applikation müsste eine Fehlerbehandlung für die Ratenlimits (Twitter (o. J.):
Rate limits) implementiert werden, sodass der Grund für ein Fehlverhalten klar ersichtlich wird. Die Limits sind nach
dem spezifischen Endpoint und nach Art der HTTP-Requests, genauer den GET- und POST-Methoden gegliedert. Die
verschiedenen Endpoints unterscheiden sich im Zeitfenster und in der Anzahl Anfragen, welche in dieser Zeit pro
Applikation getätigt werden können. Für die GET-Endpoints beträgt ein Zeitfenster stets 15 Minuten. Für POST-Endpoints
unterscheidet sich dies ferner. Als Beispiel ist nachfolgend der POST-Endpoint favorites/create abgebildet:

![Post-Endpoint](images/POST-Endpoint_example.png)

Für alle weiteren Endpoints wird auf die offizielle Twitter Entwicklerdokumentation (Twitter (o. J.): Rate limits)
verwiesen, da eine komplette Auflistung hier den Rahmen sprengen würde. Der aktuelle Stand der Limits seine eigene
Applikation betreffend lässt sich über einen GET-Endpoint (Twitter (o. J.): GET application/rate_limit_status) abfragen.

Eine Anfrage eines Client wird vom Load Balancer entgegengenommen. Es sind mehrere solcher Load Balancer vorhanden die
sich in verschiedene Regionen aufteilen. Der Load Balancer sendet die Anfrage an ein REDIS Cluster. Dort wird nun ein
sogenanntes Fan-out gemacht. Das bedeutet, die Anfrage wird in eine In-Memory Datenbank gespeichert und im Hintergrund
werden alle betroffenen Home Timelines angepasst. Gleichzeitig wird parallel der Tweet abgespeichert, damit dieser auch
über die Suchfunktion von Nicht-Followern gefunden wird. Bei Benutzern mit mehreren Millionen Followern ist dieses
vorgehen jedoch nicht geeignet da das Anpassen von so vielen Home Timelines zu viel Zeit beanspruchen würde. Hier wird
die Home Timeline erst beim Aufruf durch den Benutzer aktualisiert.

![Twitter System Design](images/Twitter_System_Design.png)

Damit Twitters Architektur diese riesige Menge an Daten effizient persistieren, analysieren und verarbeiten kann, muss
im Endeffekt die Datenbank den Anforderungen gerecht werden. Im Folgenden werden einige Themen angeschnitten, um die
Viabilität von NoSQL-Datenbanken einzuschätzen.

### Flexible Skalierbarkeit

Damit ein Unternehmen sich mit dem schnellen Lauf der Technologie agil weiterbewegen kann, ist Flexibilität von grosser
Bedeutung. NoSQL-Datenbanken weisen diese Eigenschaft auf und sprechen somit dafür.

### Grosse Datenmengen
Twitter verfügt bereits über eine riesige Menge an Daten, welche permanent abgefragt, verändert und erweitert wird.
Klassische Datenbankmodelle wachsen zwar langsam an diese Anforderungen heran, dennoch ist NoSQL klar die präferierte
Lösung.

### Analysierbarkeit

Der primäre Nutzen von Twitter stellt die Analysierbarkeit der Daten dar. Dies ist jedoch nur bei einer passender
Struktur möglich. Klassische Datenbanken erfüllen dieses Kriterium bereits seit ihrer Entstehung, wobei NoSQL dies
nicht zwingend erzwingt. Bei einem unüberlegten Datenmodell und stetiger Umgestaltung kann hierdurch ein Wildwuchs
entstehen, welcher die Untersuchung beeinträchtigt. Aus diesem Grund ist es von hoher Bedeutung auch im NoSQL eine gut
durchdachte Struktur der Daten zu wählen.

## Fazit / Konklusion / Schlussfolgerung / Diskussion

Bei prominenten Stichwörtern sind die Abstände für neue Tweets sehr kurz. Dadurch lassen sich die riesigen Tweet-Mengen
die Twitter pro Sekunde erhält erahnen und zu einem kleinen Teil sichtbar machen.

Wir sehen, dass Twitter verschiedene Herausforderungen meistern muss, um mit den riesigen Anfrage-Mengen umgehen zu
können. Die Speicherkapazität steht dabei nicht im Vordergrund, da die einzelnen Textnachrichten lediglich 280 Zeichen
umfassen können. Hingegen sind die Abhängigkeiten zwischen Benutzern und Tweets wichtig, denn gerade bei Benutzern mit
sehr vielen Followern müssen im Hintergrund viele Abhängigkeiten zu anderen Timelines angepasst werden. Vermutlich könnte
dies mit klassischen, relationalen SQL-Datenbanken schlechter bewerkstelligt werden, als mit NoSQL-Datenbanken.
Für den NoSQL-Ansatz muss jedoch eine einheitliche Datenstruktur unter den Dokumenten etabliert werden, um die
Analysierbarkeit zu gewährleisten. Den effektiven Einsatz von NoSQL hängt zudem vom jeweiligen Gebiet ab. So werden
gemäss einer Präsentation (vgl. Weil, Kevin (2010): NoSQL at Twitter) von Kevin Weil mehrere Datenbankmodelle
gleichzeitig geführt. Zudem bedeutet eine Umstrukturierung immer eine Investition von Ressourcen, was nicht unbedingt
risikofrei und ökonomisch sinnvoll sein muss.

---
## Gloassar

#### API
Application Programming Interface ist eine Programmierschnittstelle um die Kommunikation zwischen Diensten zu
erlauben.

#### Benutzer
Bei Twitter können auch nicht reale Personen einen Account besitzen wie z.B.(Firmen, Imaginäre Personen, Bands)

#### Follower
Man kann anderen Nutzern folgen (follow) und erhält dadurch Ihre Tweets.

#### NoSQL
Eine Strukturform der Datenmodelle einer Datenbank in einer anderen Weise als die tabullare Charakteristik von
klassischen, relationalen Datenbanken.

#### Timelines
Die ​ Home Timeline ist beinhaltet alle Tweets von Benutzern, denen man folgt.
Die User TImeline​ , ist die eigene Benutzerspezifische Ansicht, die die eigens erstellten Tweets anzeigt.

#### Tweet
Mit einem Tweet ​ ist eine Kurznachricht gemeint, die maximal 140 Zeichen lang ist.

---
## Quellenverzeichnis

### Literaturverzeichnis

[1] L., Narendra (2018): System design for Twitter - Narendra L, in: ​ Medium ​ , [online]
https://medium.com/@narengowda/system-design-for-twitter-e737284afc95 [05.09.2020].

[2] Russell, Matthew A. / Mikhail Klassen (2019): Mining the Social Web: Data Mining Facebook, Twitter, LinkedIn, Instagram,
GitHub, and More, 3. Aufl., Sebastopol, USA, California: O’Reilly Media.

[3] Success in Tech (2017): System Design: How to design Twitter? Interview question at Facebook, Google, Microsoft,
in: ​ YouTube , ​ [online] https://www.youtube.com/watch?v=KmAyPUv9gOY [05.09.2020].

[4] Tech Dummies (2018): Twitter system design | twitter Software architecture | twitter interview questions,
in: ​ YouTube , ​ [online] https://www.youtube.com/watch?v=wYk0xPP_P_8 [05.09.2020].

[5] Twitter (o. J.): Developer Agreement and Policy – Twitter Developers, in: ​ Twitter Developer ​ , [online]
https://developer.twitter.com/en/developer-terms/agreement-and-policy [05.09.2020a].

[6] Twitter (o. J.): GET application/rate_limit_status, in: ​ Docs | Twitter Developer ​ , [online]
https://developer.twitter.com/en/docs/twitter-api/v1/developer-utilities/rate-limit-sta tus/api-reference/get-application-rate_limit_status [05.09.2020b].

[7] Twitter (o. J.): Rate limits, in: ​ Docs | Twitter Developer , ​ [online]
https://developer.twitter.com/en/docs/twitter-api/v1/rate-limits [05.09.2020c].

[8] Twitter (o. J.): Twitter API v1.1, in: Docs | Twitter Developer, [online]
https://developer.twitter.com/en/docs/twitter-api/v1 [05.09.2020d].

[9] Weil, Kevin (2010): NoSQL at Twitter, in: ​ InfoQ , ​ [online]
https://www.infoq.com/presentations/NoSQL-at-Twitter/ [05.09.2020].